In [1]:
import pandas as pd
import numpy as np
import glob
import plotly.graph_objects as go

In [2]:
# functions written by Mitali and Girish in 2024
# adapted by Tanvi Deora, April 2025

def theta_phi(vector1, vector2):
    """
    Calculate the angles theta and phi between two vectors
    Inputs:
        The two vectors in dataframe format n * 3, x y z axis in columns
        ab - heading vector of the fly (using subsequent - t-1, t0 and t+1 - locations at all timepoints)
        cd - vector between chaser and chasee (using midpoins of chaser and chasee at all timepoints)

    Returns:
        theta (float) : The angle (in degrees) between point1, point 2 and point3 in the XY plane.
        phi (float) : The angle (in degrees) between point1, point 2 and point3 in the ZY plane.

    Dependencies:
        - numpy (np): Required for numerical computations and array operations.
    """
    theta = []
    phi = []
    for (index1, ab), (index2, cd) in zip(vector1.iterrows(), vector2.iterrows()):
        # Calculate the theta angle (angle in XY plane)
        tt = np.degrees(np.arctan2(np.cross(ab[:2], cd[:2]), 
                                      np.dot(ab[:2], cd[:2])))
        theta.append(tt)

        # Calculate the phi angle (angle in XZ plane) 
        ph = np.degrees(np.arctan2(np.cross(ab[[0, 2]], cd[[0, 2]]), 
                                    np.dot(ab[[0, 2]], cd[[0, 2]])))
        phi.append(ph)
    return theta, phi

In [3]:
# files = glob.glob('./Data/Outputs/Sampled_1k/COM/*_smoothened_*COM.csv')
files = glob.glob('../dataFolders/MuscaChasingMusca/Outputs/Sampled_1k/COM/*.csv')
files

['../dataFolders/MuscaChasingMusca/Outputs/Sampled_1k/COM\\2023-04-19_Trial7_Set1_5k_smoothened_interpol_COM.csv',
 '../dataFolders/MuscaChasingMusca/Outputs/Sampled_1k/COM\\2023-04-21_Trial6_1k_smoothened_interpol_COM.csv',
 '../dataFolders/MuscaChasingMusca/Outputs/Sampled_1k/COM\\2023-04-21_Trial7_1k_smoothened_interpol_COM.csv',
 '../dataFolders/MuscaChasingMusca/Outputs/Sampled_1k/COM\\2023-04-28_Trial4_5k_smoothened_interpol_COM.csv',
 '../dataFolders/MuscaChasingMusca/Outputs/Sampled_1k/COM\\2023-04-28_Trial5_5k_smoothened_interpol_COM.csv',
 '../dataFolders/MuscaChasingMusca/Outputs/Sampled_1k/COM\\2023-04-28_Trial8_1k_smoothened_interpol_COM.csv']

In [4]:
for f in files:
#     name = f.split('\\')[-1][:-4]
    name = f.split('\\')[-1][:-28]
    df = pd.read_csv(f)
    
    framerate = 1000 # for sequences sampled at 1000fps
    
#     framerate = [row['framerate'] for ind, row in info.iterrows() if row['filename'][:-10] in f.split('\\')[-1]][0] 
    
    # get points to compute error angles
    
    # point connecting the the chaser and chasee
    
    chasee_COM = df.loc[:, ['chasee_com_x', 'chasee_com_y', 'chasee_com_z']]
    chaser_COM = df.loc[:, ['chaser_com_x', 'chaser_com_y', 'chaser_com_z']]
    
    # ignoring the first datapoint
    vector_chaserHeading = chaser_COM.iloc[1:,:].reset_index(drop=True) - chaser_COM.iloc[0:-1,:].reset_index(drop=True)
    
    midpoint_chasee = (chasee_COM.iloc[0:-1,:].reset_index(drop=True) + chasee_COM.iloc[1:,:].reset_index(drop=True))/2
    midpoint_chasee.columns = ['x', 'y', 'z']
    midpoint_chaser = (chaser_COM.iloc[0:-1,:].reset_index(drop=True) + chaser_COM.iloc[1:,:].reset_index(drop=True))/2
    midpoint_chaser.columns = ['x', 'y', 'z']
    
    vector_chaserChasee = midpoint_chasee - midpoint_chaser.values
    
    # compute error angle in degree
    error_theta, error_phi = theta_phi(vector_chaserHeading, vector_chaserChasee)
    # compute turning angle for chaser
    turning_theta, turning_phi = theta_phi(vector_chaserHeading.iloc[1:, :], vector_chaserHeading.iloc[0:-1, :])
    turning_theta.insert(0, np.nan)
    turning_phi.insert(0,np.nan)
    
    chase_df = pd.DataFrame({'error_theta': error_theta, 
                            'error_phi': error_phi,
                            'turning_theta': turning_theta,
                            'turning_phi':turning_phi})

    chase_df.to_csv('../dataFolders/MuscaChasingMusca/Outputs/Sampled_1k/ChaseParameters_v1/' + name + '_ChaseErrorAngle_TurningAngle.csv')